In [ ]:
import pandas as pd
import numpy as np
import datetime
import re
from datetime import timedelta
from datetime import date


In [ ]:
# cargar los datos del archivo Excel
df = pd.read_excel('/content/drive/MyDrive/Repositorio/FacturacionSSTT/Facturacion historica.xlsx', sheet_name='Repositorio FST', na_values=[0])

nuevos_nombres = ["oficina","tipo_orden_trabajo","numero_orden_trabajo","tipo_trabajo","numero_documento_venta","fecha_venta","numero_siniestro","numero_orden","numero_correlacion","codigo_recepcionista","agente_confirmo_7d","fecha_apertura_orden_trabajo","fecha_termino_orden_trabajo","detalle_factura","rut_cliente","nombre_cliente","patente","marca","modelo","total_costo","total_neto_mano_obra","total_neto_repuestos","total_neto_accesorios","total_neto_lubricantes","total_neto_materiales","total_neto_otros","total_neto_documento","total_bruto_documento","cantidad_litros_lubricantes","forma_pago","vin","tipo_venta","tipo_mantenimiento","kilometros_vehiculo","posee_mpp","id_agente","numero_presupuesto", "destinatario_kovcas"]
df.columns = nuevos_nombres 

df["numero_siniestro"].fillna(0, inplace=True)


# eliminar los registros donde la columna numero_siniestro es NaN o None
df = df[df['numero_siniestro'].notna()]

# convertir la columna numero_siniestro de tipo float a int
df['numero_siniestro'] = df['numero_siniestro'].astype(int)

df_ = df.copy()
df.shape


### PASOS PARA LA NORMALIZACION DE DATA

In [ ]:
#Se crea columna contiene obd y se pasa a mayusculas lo que es detalle_factura
df= df.assign(total_ingresos = ((df['total_neto_accesorios'] + df['total_neto_lubricantes'] + df['total_neto_materiales'] + df['total_neto_repuestos']) - df['total_costo']) + df['total_neto_mano_obra'])

In [ ]:
df['total_facturacion'] = df['total_neto_accesorios'] + df['total_neto_lubricantes'] + df['total_neto_materiales'] + df['total_neto_repuestos'] + df['total_neto_mano_obra']

In [ ]:
df['año_venta'] = pd.DatetimeIndex(df['fecha_venta']).year

In [ ]:
df['mes_venta'] = pd.DatetimeIndex(df['fecha_venta']).month

In [ ]:
df['detalle_factura'].fillna('N/A', inplace=True)

df['contiene_obd'] = np.where(df['detalle_factura'].str.contains("OBD|ANTICLO", na=False), 'SI', 'NO')

In [ ]:
#Se Agrega La Columna Nota Credito
df['nota_credito'] = np.where(df['total_neto_repuestos'] >= 0, 'SIN NC', 'CON NC')

In [ ]:
df['oficina'] = df['oficina'].str.upper()
def asignar_unidad_negocios(oficina):
    if "NISSAN" in oficina:
        return "NISSAN"
    elif "TOYOTA" in oficina:
        return "TOYOTA"
    elif "DYP" in oficina:
        return "DYP"
    else:
        return "CHEVROLET"

df["unidad_negocios"] = df["oficina"].apply(asignar_unidad_negocios)

In [ ]:
#Agregar columna tipo cliente
# df['tipo_cliente'] = df['nombre_cliente'].apply(lambda x: 'SEGURO' if 'SEGURO' in str(x) else 'GENERAL')

df['tipo_cliente'] = df['nombre_cliente'].apply(lambda x: 'SEGURO' if ('SEGURO' in str(x) or 'ASEGURADORA' in str(x)) else 'GENERAL')


In [ ]:
#Cambiar tipificacion de acuerdo a condicion
df.loc[df['posee_mpp'] == 'SI', 'tipo_trabajo'] = 'Mant. Prev.'

In [ ]:
# Se re tipifica la columna Tipo Trabajo
df['tipo_trabajo'] = df['tipo_trabajo'].replace(["Acc. u otros", "Accesorios", "Corr.,Acc. u otros"], "CORRECTIVA")
df['tipo_trabajo'] = df['tipo_trabajo'].replace(["Mant. Prev."], "PREVENTIVA")
df['tipo_trabajo'] = df['tipo_trabajo'].replace(["CRÉDITO"], "CREDITO")
df['tipo_trabajo'] = df['tipo_trabajo'].replace(["GARANTÍA"], "GARANTIA")

In [ ]:
# Se reemplan todos los Valores en la columna tiene_mpp de SI a NO en caso que el registro tenga MPP 
df.loc[df['contiene_obd'] == 'SI', 'posee_mpp'] = 'NO' 

In [ ]:
# Se reemplan todos los Valores en la columna tiene_mpp de SI a NO en caso que el registro tenga MPP 
# df.loc[df['contiene_obd'] == 'SI', 'posee_mpp'] = 'NO' 

# Se reemplan todos los Valores en la columna tiene_mpp de SI a NO en caso que el registro tenga MPP 
df.loc[df['contiene_obd'] == 'SI', 'tipo_trabajo'] = 'CORRECTIVA' 

In [ ]:
#Se normaliza la data en la columna *Tipo Trabajo*
df['tipo_trabajo'] = df['tipo_trabajo'].str.replace('Á', 'A').str.replace('É', 'E').str.replace('Í', 'I').str.replace('Ó', 'O').str.replace('Ú', 'U').str.upper()
df['tipo_trabajo'] = df['tipo_trabajo'].replace(np.nan, 'sin_tipo_trabajo', regex=True)

In [ ]:
#Se normaliza la data en la columna *Tipo Trabajo*
df['tipo_orden_trabajo'] = df['tipo_orden_trabajo'].str.replace('Á', 'A').str.replace('É', 'E').str.replace('Í', 'I').str.replace('Ó', 'O').str.replace('Ú', 'U').str.upper()
df['tipo_orden_trabajo'] = df['tipo_orden_trabajo'].replace(np.nan, 'sin_orden_trabajo', regex=True)

In [ ]:
# Se normaliza la data en la columna Tipo Orden Trabajo
df['tipo_orden_trabajo'].fillna('N/A', inplace=True)
df['tipo_orden_trabajo'] = df['tipo_orden_trabajo'].str.replace('Á', 'A').str.replace('É', 'E').str.replace('Í', 'I').str.replace('Ó', 'O').str.replace('Ú', 'U').str.upper()
df['tipo_orden_trabajo'] = df['tipo_orden_trabajo'].replace(np.nan, 'sin_tipo_orden_trabajo', regex=True)

In [ ]:
# Se genera una nueva columna con TIPO VENTA OT
df["tipo_venta_ot"] = np.where(df["tipo_venta"].str.contains(re.compile("REPTOS|ANT.RPTO|CUP. MPP|RENTACAR|ANT.REPA|CHATARRA|GRUA|Deduc|CAFETERÍ|MISCELÁN",re.IGNORECASE)), "NO", "SI")

In [ ]:
#Se Reemplaza los valores del tipo venta OT
df['tipo_venta_ot'] = df['tipo_venta_ot'].replace({'SI': 'VENTAS CON OT', 'NO': 'OTRAS VENTAS'})

In [ ]:
#Se reemplaza los valores de la columna tipo Orden de Trabajo
df['tipo_orden_trabajo'] = df['tipo_orden_trabajo'].replace({'SIN_TIPO_ORDEN_TRABAJO': 'OTRAS VENTAS'})

In [ ]:
df['tipo_orden_trabajo'].replace('SIN_ORDEN_TRABAJO', 'MISCELANEA', inplace=True)
df['tipo_trabajo'].replace('sin_tipo_trabajo', 'MISCELANEA', inplace=True)

In [ ]:

df.loc[(df['contiene_obd'] == 'SI') & ((df['tipo_orden_trabajo'] == 'GARANTIA') | (df['tipo_orden_trabajo'] == 'PREENTREGA')), 'tipo_orden_trabajo'] = 'MECANICA' 

In [ ]:
#Contar OT
df['contar_ot'] = np.where(df['tipo_venta_ot'] == 'VENTAS CON OT', 1, 0)


In [ ]:
df['vin'] = df['vin'].astype(str)

def clean_vin(vin):
    if len(vin) != 17:
        return ''
    return vin

df['vin'] = df['vin'].apply(clean_vin)

In [ ]:
df['patente'] = df['patente'].astype(str)

def clean_patente(patente):
    if len(patente) != 7:
        return ''
    return patente

df['patente'] = df['patente'].apply(clean_patente)

In [ ]:
df['vin_patente'] = np.where(df['vin'] == "", df['patente'], df['vin'])

In [ ]:
df['numero_siniestro'] = df['numero_siniestro'].astype(str)

df = df[~df['numero_siniestro'].astype(str).str.contains('^0$')]

df['siniestro_patente'] = df['numero_siniestro'].astype(str) + df['patente']

### SEGMENTAR POR SINIESTRO

In [ ]:

# Filtrar los registros con tipo_cliente igual a "SEGURO" o "GENERAL"
df_filtrado = df[df.tipo_cliente.isin(["SEGURO", "GENERAL"])]

# Convertir la columna fecha_venta a tipo datetime
df_filtrado['fecha_venta'] = pd.to_datetime(df_filtrado['fecha_venta'])

# Ordenar los registros por siniestro_patente y fecha_venta descendente
df_filtrado = df_filtrado.sort_values(by=['siniestro_patente', 'fecha_venta'], ascending=[True, False])

# Mantener todos los registros con el mismo siniestro_patente
df_agrupado = df_filtrado.groupby('siniestro_patente').apply(lambda x: x)



df_agrupado[['total_ingresos', 'total_facturacion', 'total_costo', 'total_neto_mano_obra', 'total_neto_repuestos', 'total_neto_accesorios', 'total_neto_lubricantes', 'total_neto_materiales', 'total_neto_otros', 'total_neto_documento', 'total_bruto_documento']] = df_agrupado[['total_ingresos', 'total_facturacion', 'total_costo', 'total_neto_mano_obra', 'total_neto_repuestos', 'total_neto_accesorios', 'total_neto_lubricantes', 'total_neto_materiales', 'total_neto_otros', 'total_neto_documento', 'total_bruto_documento']].apply(pd.to_numeric)

df_agrupado[['total_ingresos', 'total_facturacion', 'total_costo', 'total_neto_mano_obra', 'total_neto_repuestos', 'total_neto_accesorios', 'total_neto_lubricantes', 'total_neto_materiales', 'total_neto_otros', 'total_neto_documento', 'total_bruto_documento']] = df_agrupado.groupby('siniestro_patente')[['total_ingresos', 'total_facturacion', 'total_costo', 'total_neto_mano_obra', 'total_neto_repuestos', 'total_neto_accesorios', 'total_neto_lubricantes', 'total_neto_materiales', 'total_neto_otros', 'total_neto_documento', 'total_bruto_documento']].transform('sum')



# Seleccionar el registro con tipo_cliente igual a "SEGURO" y la fecha_venta más reciente
df_agrupado = df_agrupado[(df_agrupado.tipo_cliente == 'SEGURO')].sort_values(by=['siniestro_patente', 'fecha_venta'], ascending=[True, False]).drop_duplicates(subset='siniestro_patente', keep='first')

# Seleccionar las columnas necesarias
df_agrupado = df_agrupado[['oficina', 'tipo_orden_trabajo', 'tipo_trabajo', 'fecha_venta', 'numero_siniestro', 'numero_orden', 'fecha_apertura_orden_trabajo', 'fecha_termino_orden_trabajo', 'rut_cliente', 'nombre_cliente', 'patente', 'marca', 'modelo', 'forma_pago', 'vin', 'tipo_mantenimiento', 'posee_mpp', 'contiene_obd', 'nota_credito', 'unidad_negocios', 'tipo_cliente', 'contar_ot', 'vin_patente', 'siniestro_patente', 'total_ingresos', 'total_facturacion', 'total_costo', 'total_neto_mano_obra', 'total_neto_repuestos', 'total_neto_accesorios', 'total_neto_lubricantes', 'total_neto_materiales', 'total_neto_otros', 'total_neto_documento', 'total_bruto_documento']]


In [ ]:

df_agrupado_general = df_filtrado.groupby('siniestro_patente').apply(lambda x: x)



df_agrupado_general[['total_ingresos', 'total_facturacion', 'total_costo', 'total_neto_mano_obra', 'total_neto_repuestos', 'total_neto_accesorios', 'total_neto_lubricantes', 'total_neto_materiales', 'total_neto_otros', 'total_neto_documento', 'total_bruto_documento']] = df_agrupado_general[['total_ingresos', 'total_facturacion', 'total_costo', 'total_neto_mano_obra', 'total_neto_repuestos', 'total_neto_accesorios', 'total_neto_lubricantes', 'total_neto_materiales', 'total_neto_otros', 'total_neto_documento', 'total_bruto_documento']].apply(pd.to_numeric)

df_agrupado_general[['total_ingresos', 'total_facturacion', 'total_costo', 'total_neto_mano_obra', 'total_neto_repuestos', 'total_neto_accesorios', 'total_neto_lubricantes', 'total_neto_materiales', 'total_neto_otros', 'total_neto_documento', 'total_bruto_documento']] = df_agrupado_general.groupby('siniestro_patente')[['total_ingresos', 'total_facturacion', 'total_costo', 'total_neto_mano_obra', 'total_neto_repuestos', 'total_neto_accesorios', 'total_neto_lubricantes', 'total_neto_materiales', 'total_neto_otros', 'total_neto_documento', 'total_bruto_documento']].transform('sum')



# Seleccionar el registro con tipo_cliente igual a "SEGURO" y la fecha_venta más reciente
df_agrupado_general = df_agrupado_general[(df_agrupado_general.tipo_cliente == 'GENERAL')].sort_values(by=['siniestro_patente', 'fecha_venta'], ascending=[True, False]).drop_duplicates(subset='siniestro_patente', keep='first')

# Seleccionar las columnas necesarias
df_agrupado_general = df_agrupado_general[['oficina', 'tipo_orden_trabajo', 'tipo_trabajo', 'fecha_venta', 'numero_siniestro', 'numero_orden', 'fecha_apertura_orden_trabajo', 'fecha_termino_orden_trabajo', 'rut_cliente', 'nombre_cliente', 'patente', 'marca', 'modelo', 'forma_pago', 'vin', 'tipo_mantenimiento', 'posee_mpp', 'contiene_obd', 'nota_credito', 'unidad_negocios', 'tipo_cliente', 'contar_ot', 'vin_patente', 'siniestro_patente', 'total_ingresos', 'total_facturacion', 'total_costo', 'total_neto_mano_obra', 'total_neto_repuestos', 'total_neto_accesorios', 'total_neto_lubricantes', 'total_neto_materiales', 'total_neto_otros', 'total_neto_documento', 'total_bruto_documento']]


In [ ]:
df_siniestros = pd.concat([df_agrupado, df_agrupado_general], ignore_index=True)
df_siniestros.tipo_cliente.sample(5)
df_siniestros.to_excel("SiniestroSSTT.xlsx", index=False)